# Combinar los dos set de datos

In [10]:
import os
import shutil
import cv2

Todas las rutas de entrada que tenemos para el set de datos de Roboflow.


In [11]:
train_images_folder = 'Señales-de-transito-34/train/images'
valid_images_folder = 'Señales-de-transito-34/valid/images'
test_images_folder = 'Señales-de-transito-34/test/images'

train_labels_folder = 'Señales-de-transito-34/train/labels'
valid_labels_folder = 'Señales-de-transito-34/valid/labels'
test_labels_folder = 'Señales-de-transito-34/test/labels'

Las rutas de salida donde queremos combinar los dos set de datos.

In [12]:
output_images_folder = 'datasets/cropped_and_labeled_final/images'
output_labels_folder = 'datasets/cropped_and_labeled_final/labels'

In [13]:
os.makedirs(output_images_folder, exist_ok=True)
os.makedirs(output_labels_folder, exist_ok=True)

Solamente necesitamos las clases de interes de Alto y los tipos de semáforo del set de datos de Roboflow. Estos tienen un nombre especificado en el archivo data.yml.
- 2: Pare
- 3: Semaforo peatonal rojo
- 4: Semafoto peatonal verde

In [14]:
classes_of_interest_indices = [2, 3, 4]

In [15]:
label_mapping = {
    2: "stop",  # Cambiar "Pare" a "stop sign"
    3: "pedestrian_traffic_light_red",
    4: "pedestrian_traffic_light_green"
}

In [16]:
def process_folder(images_folder, labels_folder):
    images = os.listdir(images_folder)

    for image_filename in images:
        image_path = os.path.join(images_folder, image_filename)

        # Reemplazar puntos adicionales en los nombres de archivo, si es necesario
        image_filename_cleaned = image_filename.replace('.', '_').rsplit('_', 1)[0] + '.' + image_filename.rsplit('.', 1)[1]
        label_filename = image_filename.replace('.jpg', '.txt')
        label_filename_cleaned = image_filename_cleaned.replace('.jpg', '.txt')

        label_path = os.path.join(labels_folder, label_filename)

        if not os.path.exists(label_path):
            continue  # Si no hay archivo de label, saltar

        # Leer la imagen
        img = cv2.imread(image_path)
        h, w, _ = img.shape  # Obtener el tamaño de la imagen

        with open(label_path, 'r') as label_file:
            label_data = label_file.readlines()

        # Procesar cada línea de etiquetas y verificar si contiene una clase de interés
        for line in label_data:
            parts = line.strip().split()
            label_index = int(parts[0])  # El primer número en la línea es el índice de la clase

            if label_index in classes_of_interest_indices:
                # Coordenadas YOLO: (class_id, x_center, y_center, width, height) (todas en formato relativo)
                x_center_rel = float(parts[1])
                y_center_rel = float(parts[2])
                bbox_width_rel = float(parts[3])
                bbox_height_rel = float(parts[4])

                # Convertir coordenadas relativas a absolutas (píxeles)
                x_center = int(x_center_rel * w)
                y_center = int(y_center_rel * h)
                bbox_width = int(bbox_width_rel * w)
                bbox_height = int(bbox_height_rel * h)

                # Calcular las coordenadas del bounding box
                x_min = max(0, x_center - bbox_width // 2)
                y_min = max(0, y_center - bbox_height // 2)
                x_max = min(w, x_center + bbox_width // 2)
                y_max = min(h, y_center + bbox_height // 2)

                # Recortar la imagen basada en las coordenadas del bounding box
                cropped_img = img[y_min:y_max, x_min:x_max]

                # Redimensionar la imagen recortada a 256x256
                resized_img = cv2.resize(cropped_img, (256, 256))

                # Guardar la imagen recortada en la carpeta de salida
                output_image_path = os.path.join(output_images_folder, image_filename_cleaned)
                cv2.imwrite(output_image_path, resized_img)

                # Obtener el nombre de la clase basado en el mapeo
                label_name = label_mapping[label_index]

                # Guardar la etiqueta con el nombre adecuado
                output_label_path = os.path.join(output_labels_folder, label_filename_cleaned)

                with open(output_label_path, 'w') as output_label_file:
                    output_label_file.write(label_name)

                break  # Ya hemos encontrado una clase de interés, no necesitamos seguir procesando este archivo


In [17]:
process_folder(train_images_folder, train_labels_folder)
process_folder(valid_images_folder, valid_labels_folder)
process_folder(test_images_folder, test_labels_folder)